# PART IV: CALCULATION OF COSINE SIMILARITY

## FINE-TUNED BERT

### 1. Setting up

**MODULE**

In [1]:
# Load nessesary libraries.
from transformers import pipeline, BertForSequenceClassification, AutoTokenizer
import pandas as pd
import torch
import torch.nn.functional as F
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import re
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

**GPU**

In [2]:
# Check whether CUDA is accessible and, if so, create a CUDA device.
cuda_available = torch.cuda.is_available()
cuda_device= torch.cuda.get_device_name(0)

if cuda_available == True:
    device = torch.device('cuda')
    print('CUDA was successfully installed and compiled on my device.')
    print('CUDA device name is:', cuda_device)
else:
    print('Cuda in not available')

CUDA was successfully installed and compiled on my device.
CUDA device name is: NVIDIA GeForce GTX 1650


**DATASET**

*job seekers'*

In [7]:
# Load the experiment participants dataset and apply minor modifications for further use.
df_jobseeker = pd.read_csv('data_jobseeker.csv', index_col=None)

df_jobseeker['professional_qualifications'] = df_jobseeker.education + ". " + df_jobseeker.skill + ". " + df_jobseeker.experience + '.'
df_jobseeker.drop(['education', 'skill', 'experience'], axis=1, inplace=True)

df_jobseeker.head(3)

,participant,data_collection,date,location,preferred_position,professional_qualifications
0,user_1,voice call,2023-12-17 15:30:00,"dublin, ireland",registered nurse,bachelor's degree: critical care nursing. pati...
1,user_2,voice call,2023-12-27 11:50:00,"dublin, ireland",electrician,"high school diploma, vocational electrician ce..."
2,user_3,google form,2023-12-31 13:39:00,"dublin, ireland",data analyst,"degree: master of science in data analytics, b..."


In [8]:
# Load the online job ads dataset and apply minor modifications for further use.
df_jobads = pd.read_csv('data_jobads_final.csv', index_col=None)

df_jobads['job_description'] = df_jobads['job_description'].str.replace('\n', ' ')
df_jobads = df_jobads.dropna()
df_jobads = df_jobads.reset_index(drop=True)

df_jobads.head(2)

,title,id,link,date,job_description,label
0,assistant director of nursing,sj_3c7e64c7996bb9d6,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"January 10, 2024",silver stream healthcare group offer great emp...,registered_nurse
1,clinical nurse manager (cnm),sj_358f1f68cde928c4,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,unknown,create a better future for yourself recruitne...,registered_nurse


### 2. Testing the classification model and checking the model's embedding.

**Initializing MODEL**

Initializing a fine-tuned BERT model for classification tasks and retrieving encoded results.

In [ ]:
# Initialize a fine-tuned model with the hidden state output enabled.
model = BertForSequenceClassification.from_pretrained('ft_bert_temuulen2', output_hidden_states=True)

# Initialize a tokenizer used for the fine-tuned model.
tokenizer = AutoTokenizer.from_pretrained('ft_bert_temuulen_tokenizer2')

# Initialize a text classification pipeline.
classifier = pipeline('text-classification', model='ft_bert_temuulen2', tokenizer='ft_bert_temuulen_tokenizer2')

**CLASSIFICATION TEST**

In [49]:
# Test model with random text samples.
random_text = ["1. I promote health and help people who is sick", 
               "2. If your outlet isn't working and you don't have a light, you can ask me to fix it.", 
               "3. All day, I'm sitting in front of the screen, solving problems with my mouse and keyboard", 
               "4. I'm familiar with the process of providing injections to individuals.",
               "5. If the streetlights fail, I can replace them with new ones.",
               "6. I'm familiar with both Python and R, and I use these tools for my work."]

for x in range(len(random_text)):
    print(classifier(random_text[x]), '=', random_text[x])

[{'label': 'registered_nurse', 'score': 0.9776366353034973}] = 1. I promote health and help people who is sick
[{'label': 'electrician', 'score': 0.9333652257919312}] = 2. If your outlet isn't working and you don't have a light, you can ask me to fix it.
[{'label': 'data_analyst', 'score': 0.8441108465194702}] = 3. All day, I'm sitting in front of the screen, solving problems with my mouse and keyboard
[{'label': 'registered_nurse', 'score': 0.9447991251945496}] = 4. I'm familiar with the process of providing injections to individuals.
[{'label': 'electrician', 'score': 0.9593295454978943}] = 5. If the streetlights fail, I can replace them with new ones.
[{'label': 'data_analyst', 'score': 0.9338756203651428}] = 6. I'm familiar with both Python and R, and I use these tools for my work.


The random sentences employed to evaluate the fine-tuned Bert-base classification model were intentionally vague and concise, aiming to assess the model's capability in challenging scenarios. Nevertheless, the outcomes and associated scores demonstrate that the model excelled, accurately forecasting every sentence with significant confidence levels (average probability score: **0.9322**).

In [39]:
# Test model with job seekers' data.
for x in range(3):
    print(classifier(df_jobseeker.iat[x, -1]), '=', df_jobseeker.iat[x, -1])

[{'label': 'registered_nurse', 'score': 0.9979069232940674}] = bachelor's degree: critical care nursing. patient care, wound care, medical procedures, adult nursing, infection control, diagnostic, time management, communication skills, attention to detail. registered nurse: 3 years.
[{'label': 'electrician', 'score': 0.9958240985870361}] = high school diploma, vocational electrician certification, construction safety certification. circuit testing, blueprint reading, fault finding, electrical wiring, troubleshooting, equipment inspection, installation, organization, maintenance, diagnostic, independent worker, safety knowledge. residential electrician's helper: 1 year.
[{'label': 'data_analyst', 'score': 0.996567964553833}] = degree: master of science in data analytics, bachelor of science in business administration; certifications: microsoft certified - azure data scientist associate, google data analytics certificate. python, data mining and extraction, data analytics and visualizati

All data from each participant in the experiment was classified and assigned into the appropriate labelled classes with outstanding results.(average probability score: **0.9968**)

**ENCODING TEST**

Check the encoder output of the model using one of the job seeker data

In [7]:
# Choose text for testing.
input_text = df_jobseeker.iat[0, -1]

# Tokenize the input text and convert it to PyTorch tensors.
inputs = tokenizer(input_text, return_tensors='pt')

# Perform a forward pass through the model to get the hidden states.
with torch.no_grad():
    outputs = model(**inputs)

# Extract the last hidden states from the model outputs.
last_hidden_states = outputs.hidden_states[-1]

print(last_hidden_states.shape, '\n')
print(last_hidden_states)

torch.Size([1, 44, 768]) 

tensor([[[ 1.0482, -0.5065,  0.3688,  ..., -0.7035, -1.5218, -0.0639],
         [ 1.1028, -0.3971,  0.3137,  ..., -0.4876, -0.9391, -0.4248],
         [ 1.3280,  0.2820,  0.2279,  ..., -0.5051, -1.3504,  0.0178],
         ...,
         [ 0.6896,  0.7037,  0.8631,  ..., -0.7715, -1.0878, -0.6608],
         [ 0.2062, -0.3984,  0.5171,  ...,  0.1593, -1.0404, -0.6125],
         [ 1.2090,  0.1183,  0.2903,  ..., -0.3679, -1.5123, -0.0687]]])


### 3. Embedding

In [10]:
# Define a custom function to extract the final layer encodings from BERT, without conditions.
def process_text(text):
    
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt')
    
    # Pass the tokenized input through the model.
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Retrieve the last hidden states from the model's outputs.
    last_hidden_states = outputs.hidden_states[-1]
    
    return last_hidden_states

In [ ]:
# Define a custom function to extract the final layer encodings from BERT, with conditions.
def embed_with_bert(df_column):
    
    embedded_texts = []
    
    # Iterate through each text in the DataFrame column.
    for text in df_column:
        
        # Tokenize each text without adding special tokens and without truncation or padding.
        tokens = tokenizer(text, add_special_tokens=False, return_tensors='pt', truncation=False, padding=False)['input_ids'].squeeze()
        token_length = len(tokens)
        
        # If the token length is less than or equal to 512, process it normally.
        if token_length <= 512:
            inputs = tokenizer(text, return_tensors='pt').to(device)
            with torch.no_grad():
                outputs = model(**inputs)
            last_hidden_states = outputs.hidden_states[-1].cpu()  
            embedded_texts.append(last_hidden_states)
            
        # If the token length is greater than 512, split it into sliding windows withot lapping.
        else:
            max_length = 512
            stride = 0
            tokens = tokenizer(text, add_special_tokens=False, return_tensors='pt', truncation=False, padding=False)['input_ids'].squeeze().to(device)
            token_windows = [tokens[i:i+max_length] for i in range(0, len(tokens), max_length - stride)]
            
            all_hidden_states = []
            
            # Add special tokens (CLS and SEP) and truncate if needed.
            for window in token_windows:
                window = torch.cat([torch.tensor([tokenizer.cls_token_id], device=device), window, torch.tensor([tokenizer.sep_token_id], device=device)])
                if len(window) > max_length:
                    window = torch.cat((window[:max_length-1], torch.tensor([tokenizer.sep_token_id], device=device)))
                inputs = {'input_ids': window.unsqueeze(0)}
                with torch.no_grad():
                    outputs = model(**inputs)
                hidden_states = outputs.hidden_states[-1].cpu()  
                all_hidden_states.append(hidden_states)
            
            # Concatenate all hidden states from each sliding window.
            embedded_texts.append(torch.cat(all_hidden_states, dim=1))
            
    return embedded_texts

**job seeker**

In [9]:
# Apply function and create a new column with the extracted results.
df_jobseeker['last_layer'] = df_jobseeker.iloc[:, -1].apply(process_text)

print(df_jobseeker.iat[0, -1].shape, '\n')
print(df_jobseeker.iat[0, -1])

torch.Size([1, 44, 768]) 

tensor([[[ 1.0482, -0.5065,  0.3688,  ..., -0.7035, -1.5218, -0.0639],
         [ 1.1028, -0.3971,  0.3137,  ..., -0.4876, -0.9391, -0.4248],
         [ 1.3280,  0.2820,  0.2279,  ..., -0.5051, -1.3504,  0.0178],
         ...,
         [ 0.6896,  0.7037,  0.8631,  ..., -0.7715, -1.0878, -0.6608],
         [ 0.2062, -0.3984,  0.5171,  ...,  0.1593, -1.0404, -0.6125],
         [ 1.2090,  0.1183,  0.2903,  ..., -0.3679, -1.5123, -0.0687]]])


In [10]:
# Check the results.
df_jobseeker

,participant,data_collection,date,location,preferred_position,professional_qualifications,last_layer
0,user_1,voice call,2023-12-17 15:30:00,"dublin, ireland",registered nurse,bachelor's degree: critical care nursing. pati...,"[[[tensor(1.0482), tensor(-0.5065), tensor(0.3..."
1,user_2,voice call,2023-12-27 11:50:00,"dublin, ireland",electrician,"high school diploma, vocational electrician ce...","[[[tensor(-0.9116), tensor(0.3615), tensor(0.1..."
2,user_3,google form,2023-12-31 13:39:00,"dublin, ireland",data analyst,"degree: master of science in data analytics, b...","[[[tensor(-0.8659), tensor(-0.2938), tensor(0...."


**job ad**

In [11]:
# Read job ads' dataset.
df_jobads = pd.read_csv('data_jobads_final.csv', index_col=None)

# Apply some data cleaning and preprocessing steps.
df_jobads['job_description'] = df_jobads['job_description'].str.replace('\n', ' ')
df_jobads = df_jobads.dropna()
df_jobads = df_jobads.reset_index(drop=True)

df_jobads.head(2)

,title,id,link,date,job_description,label
0,assistant director of nursing,sj_3c7e64c7996bb9d6,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"January 10, 2024",silver stream healthcare group offer great emp...,registered_nurse
1,clinical nurse manager (cnm),sj_358f1f68cde928c4,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,unknown,create a better future for yourself recruitne...,registered_nurse


In [12]:
# Calculate the word count for each ad and add its values to a new column.
df_jobads['word_count'] = df_jobads['job_description'].apply(lambda x: len(x.split()))

print('The total number of rows with a word count over 510 is:', df_jobads[df_jobads['word_count'] > 510].shape[0])
print('The word count for the longest text is:', df_jobads.iat[303, -1])

df_jobads.head(2)

The total number of rows with a word count over 510 is: 236
The word count for the longest text is: 3145


,title,id,link,date,job_description,label,word_count
0,assistant director of nursing,sj_3c7e64c7996bb9d6,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"January 10, 2024",silver stream healthcare group offer great emp...,registered_nurse,502
1,clinical nurse manager (cnm),sj_358f1f68cde928c4,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,unknown,create a better future for yourself recruitne...,registered_nurse,231


In [13]:
# Define a custom function to encode text with BERT, including additional rules for dealing with long text.
def embed_with_bert(df_column):
    
    embedded_texts = []
    
    # Iterate through each text in the DataFrame column.
    for text in df_column:
        
        # Tokenize each text without adding special tokens and without truncation or padding.
        tokens = tokenizer(text, add_special_tokens=False, return_tensors='pt', truncation=False, padding=False)['input_ids'].squeeze()
        token_length = len(tokens)
        
        # If the token length is less than or equal to 512, process it normally.
        if token_length <= 512:
            inputs = tokenizer(text, return_tensors='pt').to(device)
            with torch.no_grad():
                outputs = model(**inputs)
            last_hidden_states = outputs.hidden_states[-1].cpu()  
            embedded_texts.append(last_hidden_states)
            
        # If the token length is greater than 512, split it into sliding windows withot lapping.
        else:
            max_length = 512
            stride = 0
            tokens = tokenizer(text, add_special_tokens=False, return_tensors='pt', truncation=False, padding=False)['input_ids'].squeeze().to(device)
            token_windows = [tokens[i:i+max_length] for i in range(0, len(tokens), max_length - stride)]
            
            all_hidden_states = []
            
            # Add special tokens (CLS and SEP) and truncate if needed.
            for window in token_windows:
                window = torch.cat([torch.tensor([tokenizer.cls_token_id], device=device), window, torch.tensor([tokenizer.sep_token_id], device=device)])
                if len(window) > max_length:
                    window = torch.cat((window[:max_length-1], torch.tensor([tokenizer.sep_token_id], device=device)))
                inputs = {'input_ids': window.unsqueeze(0)}
                with torch.no_grad():
                    outputs = model(**inputs)
                hidden_states = outputs.hidden_states[-1].cpu()  
                all_hidden_states.append(hidden_states)
            
            # Concatenate all hidden states from each sliding window.
            embedded_texts.append(torch.cat(all_hidden_states, dim=1))
            
    return embedded_texts

In [14]:
# Move the model to the GPU.
model.to(device)

# Apply the 'embed_with_bert' function to each ad.
df_jobads['embedded_job_descriptions'] = df_jobads['job_description'].apply(lambda x: embed_with_bert([x])[0])

# Check the random cell to see the results.
print(df_jobads.iat[0, -1].shape, '\n')
print(df_jobads.iat[0, -1])

Token indices sequence length is longer than the specified maximum sequence length for this model (615 > 512). Running this sequence through the model will result in indexing errors


torch.Size([1, 617, 768]) 

tensor([[[ 0.8475, -0.3499,  0.1554,  ..., -0.9477, -1.4335, -0.0909],
         [ 0.9002, -0.1678,  0.2116,  ..., -0.4531, -0.7653,  0.3844],
         [ 0.7950, -0.4283,  0.2220,  ..., -0.8476, -1.0548, -0.3064],
         ...,
         [ 0.5650, -0.1756,  0.3656,  ..., -0.1276, -0.5553,  0.3612],
         [ 0.4369,  0.2499, -0.0900,  ..., -0.2781, -1.2343, -0.3239],
         [ 0.9924,  0.7443, -0.1271,  ..., -0.4200, -0.8044, -0.0065]]])


In [15]:
# Check the Data Frame.
df_jobads.head()

,title,id,link,date,job_description,label,word_count,embedded_job_descriptions
0,assistant director of nursing,sj_3c7e64c7996bb9d6,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"January 10, 2024",silver stream healthcare group offer great emp...,registered_nurse,502,"[[[tensor(0.8475), tensor(-0.3499), tensor(0.1..."
1,clinical nurse manager (cnm),sj_358f1f68cde928c4,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,unknown,create a better future for yourself recruitne...,registered_nurse,231,"[[[tensor(0.9966), tensor(-0.3614), tensor(0.2..."
2,registered nurse,job_4e16e9830b072344,https://ie.indeed.com/rc/clk?jk=4e16e9830b0723...,"January 10, 2024","access healthcare, one of irelands leading hea...",registered_nurse,182,"[[[tensor(0.9662), tensor(-0.3073), tensor(0.2..."
3,staff nurse,job_25a417a6373967b4,https://ie.indeed.com/rc/clk?jk=25a417a6373967...,"January 10, 2024",are you a dedicated and compassionate staff nu...,registered_nurse,199,"[[[tensor(0.9799), tensor(-0.3279), tensor(0.3..."
4,clinical research nurse - cardiology (cnm2),job_303eee71cce63f3d,https://ie.indeed.com/rc/clk?jk=303eee71cce63f...,"January 10, 2024",clinical research nurse cardiology cnm2 we a...,registered_nurse,180,"[[[tensor(0.9361), tensor(-0.4320), tensor(0.2..."
...,...,...,...,...,...,...,...,...
1161,junior analyst- private debt investments - fix...,job_ce346c7bfdd6dced,https://ie.indeed.com/rc/clk?jk=ce346c7bfdd6dc...,"before December 21, 2023",the successful candidate will have exposure to...,data_analyst,442,"[[[tensor(-0.7358), tensor(-0.4381), tensor(0...."
1162,strategy analyst - tech,job_18b9089f580d5d70,https://ie.indeed.com/rc/clk?jk=18b9089f580d5d...,"before December 21, 2023",sector: fintech you will be a datadriven indiv...,data_analyst,114,"[[[tensor(-0.7738), tensor(-0.3462), tensor(0...."
1163,"financial analyst, retail",job_e6ba95f1a8debaa8,https://ie.indeed.com/rc/clk?jk=e6ba95f1a8deba...,"before December 21, 2023",our client are recognised as a market leader a...,data_analyst,187,"[[[tensor(-0.7812), tensor(-0.4904), tensor(0...."
1164,senior operations analyst (trade support),job_ac9aa90b2b36b178,https://ie.indeed.com/rc/clk?jk=ac9aa90b2b36b1...,"before December 21, 2023",the role our operations analysts are responsib...,data_analyst,191,"[[[tensor(-0.8859), tensor(-0.4103), tensor(0...."


In [16]:
# Costum function that generates the evarage cosine similarity between the user's tensor and a job ad's tensor.
def calculate_average_similarity(tensor_user, tensor_ad):
    
    # Squeeze dimensions if the tensors have a batch dimension.
    tensor_user = tensor_user.squeeze(0) if tensor_user.dim() == 3 else tensor_user
    tensor_ad = tensor_ad.squeeze(0) if tensor_ad.dim() == 3 else tensor_ad

    tensor_ad = tensor_ad.to(tensor_user.device)

    # Initialize a similarity matrix with zeros.
    similarity_matrix = torch.zeros(tensor_user.size(0), tensor_ad.size(0), device=tensor_user.device)
    
    # Calculate cosine similarity for each pair of vectors.
    for i in range(tensor_user.size(0)):
        for j in range(tensor_ad.size(0)):
            similarity_matrix[i, j] = F.cosine_similarity(tensor_user[i].unsqueeze(0), tensor_ad[j].unsqueeze(0), dim=1)
            
    # Calculate the average similarity and convert it to a Python float.
    average_similarity = torch.mean(similarity_matrix).item()
    
    return average_similarity

In [17]:
# Record the start time.
start = time.time()

# Get the user1's tensor and move it to the GPU.
tensor_user = df_jobseeker.iat[0, -1]
tensor_user = tensor_user.to(device)

# Apply the calculation of average cosine similarity function to each job ad's tensor.
df_jobads['cosine_user1'] = df_jobads.iloc[:, -1].apply(lambda x: calculate_average_similarity(tensor_user, x.to(device)))

# Record the end time.
end = time.time()

torch.cuda.empty_cache()

print(f'The calculation was completed in: {int((end - start)) // 60} minutes and {int((end - start)) % 60} seconds.')

The calculation was completed in: 103 minutes and 23 seconds.


In [19]:
# Record the start time.
start = time.time()

# Get the user2's tensor and move it to the GPU.
tensor_user = df_jobseeker.iat[1, -1]
tensor_user = tensor_user.to(device)

# Apply the calculation of average cosine similarity function to each job ad's tensor.
df_jobads['cosine_user2'] = df_jobads.iloc[:, -2].apply(lambda x: calculate_average_similarity(tensor_user, x.to(device)))

# Record the end time.
end = time.time()

torch.cuda.empty_cache()

print(f'The calculation was completed in: {int((end - start)) // 60} minutes and {int((end - start)) % 60} seconds.')

The calculation was completed in: 139 minutes and 57 seconds.


In [21]:
# Record the start time.
start = time.time()

# Get the user3's tensor and move it to the GPU.
tensor_user = df_jobseeker.iat[2, -1]
tensor_user = tensor_user.to(device)

# Apply the calculation of average cosine similarity function to each job ad's tensor.
df_jobads['cosine_user3'] = df_jobads.iloc[:, -3].apply(lambda x: calculate_average_similarity(tensor_user, x.to(device)))

# Record the end time.
end = time.time()

torch.cuda.empty_cache()

print(f'The calculation was completed in: {int((end - start)) // 60} minutes and {int((end - start)) % 60} seconds.')

The calculation was completed in: 209 minutes and 50 seconds.


In [22]:
# Drop the encoded column from the Data Frame (it takes up too much memory and is no longer needed).
df_jobads = df_jobads.drop(columns=['embedded_job_descriptions']) 

df_jobads.head(2)

,title,id,link,date,job_description,label,word_count,cosine_user1,cosine_user2,cosine_user3
0,assistant director of nursing,sj_3c7e64c7996bb9d6,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"January 10, 2024",silver stream healthcare group offer great emp...,registered_nurse,502,0.668876,-0.105210,-0.066872
1,clinical nurse manager (cnm),sj_358f1f68cde928c4,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,unknown,create a better future for yourself recruitne...,registered_nurse,231,0.695014,-0.130288,-0.068436
2,registered nurse,job_4e16e9830b072344,https://ie.indeed.com/rc/clk?jk=4e16e9830b0723...,"January 10, 2024","access healthcare, one of irelands leading hea...",registered_nurse,182,0.699538,-0.121167,-0.072141
3,staff nurse,job_25a417a6373967b4,https://ie.indeed.com/rc/clk?jk=25a417a6373967...,"January 10, 2024",are you a dedicated and compassionate staff nu...,registered_nurse,199,0.701325,-0.128592,-0.078877
4,clinical research nurse - cardiology (cnm2),job_303eee71cce63f3d,https://ie.indeed.com/rc/clk?jk=303eee71cce63f...,"January 10, 2024",clinical research nurse cardiology cnm2 we a...,registered_nurse,180,0.723658,-0.151578,-0.079200


In [24]:
df_jobads.to_csv('cosine-bert.csv', index=False)

torch.nn.functional.cosine_similarity
1. Library: PyTorch
2. Use Case: Primarily designed for deep learning applications, where data is often in the form of tensors.
3. Input: Expects PyTorch tensors as input. It computes the cosine similarity between tensors along a specified dimension.
4. Output: Returns a tensor containing the cosine similarity scores.
5. Flexibility: Offers more control over which dimensions to compute the similarity on, making it suitable for batch operations in deep learning models.
6. Example Usage:

sorted_user1 = df_jobads.sort_values(by='cosine_user1', ascending=False) #.reset_index(drop=True)
sorted_user1.head()

sorted_user2 = df_jobads.sort_values(by='cosine_user1', ascending=False) #.reset_index(drop=True)
sorted_user2.head()

sorted_user3 = df_jobads.sort_values(by='cosine_user1', ascending=False) #.reset_index(drop=True)
sorted_user3.head()